In [1]:
#Libraries
from flask import Flask,render_template,request, url_for
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
from keras.models import Sequential
from keras.layers import LSTM,Dense
#Declaring global variables
global n_features,look_back,p
look_back=1
#define a function to get data in the form of xtrain and ytrain
def get_data(data, look_back):
    x, y = [], []
    for i in range (len(data)-look_back):
        x.append(data[i:i+look_back,0])
        y.append(data[i+look_back,0])
    return np.array(x), np.array(y)
#Loading the dataset
data = pd.read_excel('DEXINUS.xls',parse_dates=True)
df = data.iloc[10:data.shape[0]]
df = df.rename(columns={'FRED Graph Observations':'Date','Unnamed: 1':'INR'})
df['Date'] = pd.to_datetime(df['Date'])
df['INR'] = pd.to_numeric(df['INR'])
df = df.dropna()
df = df[df.INR != 0]
df = df.set_index('Date')
#Taking last 3 years data to train the model
train = df.iloc[-1095:]
train = scaler.fit_transform(train)
x_train, y_train = get_data(train,look_back)
#reshaping the training set
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
n_features = x_train.shape[1]
model =Sequential()
model.add(LSTM(100,activation='relu',input_shape=(1,1)))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss = 'mse')
model.fit(x_train,y_train, epochs = 100, batch_size=27)
#Initializing the flask app
app = Flask(__name__,template_folder='templates/',static_folder='templates/')
#Routing the app for home page
@app.route('/')
def home():
    return render_template('index.html')
#Routing the app for predict page
@app.route('/predict',methods=['POST','GET'])
def predict():
    if request.method == 'POST':
        #getting p value
        p = int(request.form['days'])
        #getting a dummy test data of 5 cells and repeating the same steps as train set 
        #for first value in predicted data
        test = df.iloc[-30:]
        test = scaler.fit_transform(test)
        x_test,y_test = get_data(test,look_back)
        x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
        y_pred = model.predict(x_test)
        tempo = y_pred
        y_pred = scaler.inverse_transform(y_pred)
        y_test = np.array(y_test).reshape(-1,1)
        y_test = scaler.inverse_transform(y_test)
        data = df.iloc[-29:]
        data['Predicted_INR'] = y_pred
        #RMSE#
        last_date = data.index[-1].date()
        n_days = p
        #Defining a future dataFrame for predictions
        future_dates = []
        for day in range(n_days):
            date = (last_date + datetime.timedelta(days = day))#.isoformat()
            future_dates.append(date)
        dummy_array = np.empty((p,2))
        dummy_array[:] = np.nan
        future_dates = pd.to_datetime(future_dates)
        index = future_dates
        columns = ['INR','Predicted_INR']
        future_data = pd.DataFrame(dummy_array,columns=columns,index=index)
        #Defining the first row values
        future_data.iloc[0,0] = float(x_test[-1])
        future_data.iloc[0,1] = float(tempo[-1])
        #Predicting
        for i in range(0,n_days-1):
            future_data['INR'][i+1] = future_data['Predicted_INR'][i]
            y = np.array(float(future_data['INR'][i+1]))
            y = y.reshape(1,1,1)
            pred = model.predict(y)
            future_data['Predicted_INR'][i+1] = pred
        #Applying scalar inverse transform
        t = future_data.INR
        z = future_data.Predicted_INR
        t = np.array(t)
        t = t.reshape(t.shape[0],1)
        z = np.array(z)
        z = z.reshape(z.shape[0],1)
        t = scaler.inverse_transform(t)
        z = scaler.inverse_transform(z)
        future_data['INR'] = t
        future_data['Predicted_INR'] = z
        d = df.iloc[-30:]
        future_data = future_data.drop(['INR'],axis=1)
        #Preparing the data to be sent to the html file
        future_data = future_data.reset_index()
        future_data = future_data.rename(columns={'index':'Date'},inplace = False)
        future_data = future_data.round({'Predicted_INR':4})
        future_data['Date'] = pd.to_datetime(future_data['Date']).dt.date
        headings0 = np.array(future_data.columns)
        data0 = np.array(future_data)    
    return render_template('result.html',headings = headings0,data = data0)
#Running the app
if __name__=='__main__':
    app.run()

Epoch 1/100
41/41 [==============================] - 15s 1ms/step - loss: 0.2760
Epoch 2/100
41/41 [==============================] - 0s 1ms/step - loss: 0.1584
Epoch 3/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0630
Epoch 4/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0285
Epoch 5/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0240
Epoch 6/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0192
Epoch 7/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0159
Epoch 8/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0119
Epoch 9/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0091
Epoch 10/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 11/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 12/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 13/100
41/41 [================

41/41 [==============================] - 0s 1ms/step - loss: 3.6555e-04
Epoch 99/100
41/41 [==============================] - 0s 1ms/step - loss: 3.9420e-04
Epoch 100/100
41/41 [==============================] - 0s 1ms/step - loss: 3.8498e-04
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Sep/2021 18:46:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2021 18:46:39] "GET /templates/titleicon.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2021 18:46:44] "POST /predict HTTP/1.1" 200 -
